In [35]:
import keras
from keras.models import load_model
#from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Flatten, Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import os
import pickle
import numpy as np
import h5py
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import pandas as pd
from PIL import Image
from keras import metrics

In [9]:
input_img = Input(shape=(2048, 2048, 3))
x = Conv2D(4, (7, 7), padding='same')(input_img)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (5, 5), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

#x = Conv2D(16, (3, 3), padding='same')(x)
#x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

#x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (16,16),padding="valid",strides=16)(x)
x = Flatten()(x)
x = Dense(32)(x)
x = Dense(3)(x)
out = Activation("softmax")(x)
model = Model(input_img, out)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

weights_path = '/mnt/Doc/weight/Autoencoder/256/best_weight_0507.h5' 
assert os.path.exists(weights_path), 'Model weights not found '
f = h5py.File(weights_path)
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
weight_value_tuples = []

for k, name in enumerate(layer_names):
    if k >= 9:
        # 全結合層の重みは読み込まない
        break
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if len(weight_names):
        weight_values = [g[weight_name] for weight_name in weight_names]
        layer = model.layers[k]
        symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
        if len(weight_values) != len(symbolic_weights):
            raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +' elements.')
        weight_value_tuples += zip(symbolic_weights, weight_values)
K.batch_set_value(weight_value_tuples)
f.close()
print('Model loaded.')

#for layer in model.layers[:9]:
        #layer.trainable = False


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2048, 2048, 3)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2048, 2048, 4)     592       
_________________________________________________________________
activation_5 (Activation)    (None, 2048, 2048, 4)     0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1024, 1024, 4)     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1024, 1024, 8)     808       
_________________________________________________________________
activation_6 (Activation)    (None, 1024, 1024, 8)     0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 512, 512, 8)       0         
__________

In [10]:
modelCheckpoint = ModelCheckpoint(filepath ="/mnt/Doc/weight/classify/2019_05_02/2019_05_08.h5",
                                  monitor = "val_loss",
                                  verbose=1,
                                  save_best_only = True,
                                  save_weights_only = False,
                                  mode = "min",
                                  period =1)

In [42]:
class ImageDataGenerator(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.images = []
        self.labels = []
        self.labelsx = []
    #file_pathはメタCSVでーたのこと
    def flow_from_directory(self, file_path, classes, batch_size=8):
        blunc=np.empty((batch_size, 2048, 2048, 3), np.float32)
        blunc_l=np.empty((batch_size), np.float32)
        # LabelEncode(classをint型に変換)するためのdict\n",
        classes = {v: i for i, v in enumerate(sorted(classes))}
        file_path = pd.read_csv(file_path)
        boss_dir = "/mnt/Doc"
        while True:
            # ディレクトリから画像のパスを取り出す
            count = 0
            for i in range(len(file_path)):
                # 画像を読み込みRGBへの変換、Numpyへの変換を行い、配列(self.iamges)に格納
                path = file_path.loc[i]["path1"]
                #print(i)

                with Image.open(path) as f:
                    self.images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)
                
                # ファイル名からラベルを取り出し、配列(self.labels)に格納
                #_, y = path.stem.split('_')
                label = file_path.loc[i]["type"]
                self.labels.append(to_categorical(classes[label],len(classes)))
                self.labelsx.append(to_categorical(classes[label],len(classes)))
                # ここまでを繰り返し行い、batch_sizeの数だけ配列(self.iamges, self.labels)に格納
                # batch_sizeの数だけ格納されたら、戻り値として返し、配列(self.iamges, self.labels)を空にする
                if len(self.images)== batch_size:
                    #print(len(self.images))
                    inputs = np.asarray(self.images,dtype=np.float32) 
                    targets = np.asarray(self.labels,dtype=np.float32)
                    #print(len(self.images))
                    count+=1
                    #if count%1000==0:
                        #print("n*1000!")
                    self.reset()
                    yield inputs, targets
   

In [12]:
#test_dir = pathlib.Path('/path/to/test/')
#x = pd.read_csv("/mnt/Doc/2048pix_tissue_all/RGBmean<190.csv")
train_dir = "/mnt/Doc/2048pix_tissue_all/data_Split_08_01_01/train_4400.csv"
test_dir = "/mnt/Doc/2048pix_tissue_all/test.csv"
train_len = len(pd.read_csv(train_dir))
print(train_len)
test_len = len(pd.read_csv(test_dir))
test_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator()
classes =["PI","PP","TRU"]
batchsize = 8
model.fit_generator(
    generator=train_datagen.flow_from_directory(train_dir, classes),
    steps_per_epoch=4400/batchsize,
    epochs=1000,
    verbose=2,
    validation_data=test_datagen.flow_from_directory(test_dir, classes),
    validation_steps=120/batchsize,
    callbacks = [modelCheckpoint]   
    )

4400
Epoch 1/1000
 - 1219s - loss: 1.1913 - acc: 0.4086 - val_loss: 1.1017 - val_acc: 0.4333

Epoch 00001: val_loss improved from inf to 1.10169, saving model to /mnt/Doc/weight/classify/2019_05_02/2019_05_08.h5
Epoch 2/1000
 - 1215s - loss: 1.0650 - acc: 0.4116 - val_loss: 1.0932 - val_acc: 0.4333

Epoch 00002: val_loss improved from 1.10169 to 1.09324, saving model to /mnt/Doc/weight/classify/2019_05_02/2019_05_08.h5
Epoch 3/1000
 - 1215s - loss: 1.0636 - acc: 0.4170 - val_loss: 1.0787 - val_acc: 0.4333

Epoch 00003: val_loss improved from 1.09324 to 1.07873, saving model to /mnt/Doc/weight/classify/2019_05_02/2019_05_08.h5
Epoch 4/1000
 - 1215s - loss: 1.0632 - acc: 0.4182 - val_loss: 1.1000 - val_acc: 0.4333

Epoch 00004: val_loss did not improve from 1.07873
Epoch 5/1000
 - 1217s - loss: 1.0622 - acc: 0.4159 - val_loss: 1.1127 - val_acc: 0.4333

Epoch 00005: val_loss did not improve from 1.07873
Epoch 6/1000
 - 1216s - loss: 1.0585 - acc: 0.4216 - val_loss: 1.0942 - val_acc: 0.425

KeyboardInterrupt: 

In [14]:
model.save_weights("/mnt/Doc/weight/weight_4400_0515.h5")

In [48]:
test_dir = "/mnt/Doc/2048pix_tissue_all/data_Split_08_01_01/val_400.csv"
test_datagen = ImageDataGenerator()
classes =["PI","PP","TRU"]
batchsize = 8
predict = model.evaluate_generator(test_datagen.flow_from_directory(test_dir,classes),
                 steps=120/batchsize,
                                 )


[0.566621499756972, 0.8166666666666667]

In [ ]:
model.evaluate_generator()

In [23]:
len(predict)

400

In [26]:
predict

array([[5.2214712e-01, 1.8164328e-01, 2.9620963e-01],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [1.1923624e-02, 4.3197232e-04, 9.8764437e-01],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [5.2214712e-01, 1.8164328e-01, 2.9620963e-01],
       [8.1604582e-01, 1.8394987e-01, 4.3246523e-06]], dtype=float32)

In [28]:
class ImageDataGenerator1(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.images = []
        self.labels = []
        self.labelsx = []
    #file_pathはメタCSVでーたのこと
    def flow_from_directory(self, file_path, classes, batch_size=8):
        blunc=np.empty((batch_size, 2048, 2048, 3), np.float32)
        blunc_l=np.empty((batch_size), np.float32)
        # LabelEncode(classをint型に変換)するためのdict\n",
        classes = {v: i for i, v in enumerate(sorted(classes))}
        file_path = pd.read_csv(file_path)
        boss_dir = "/mnt/Doc"
        while True:
            # ディレクトリから画像のパスを取り出す
            count = 0
            for i in range(len(file_path)):
                # 画像を読み込みRGBへの変換、Numpyへの変換を行い、配列(self.iamges)に格納
                path = file_path.loc[i]["path1"]
                #print(i)

                #with Image.open(path) as f:
                    #self.images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)
                
                # ファイル名からラベルを取り出し、配列(self.labels)に格納
                #_, y = path.stem.split('_')
                label = file_path.loc[i]["type"]
                #self.labels.append(to_categorical(classes[label],len(classes)))
                self.labelsx.append(to_categorical(classes[label],len(classes)))
                # ここまでを繰り返し行い、batch_sizeの数だけ配列(self.iamges, self.labels)に格納
                # batch_sizeの数だけ格納されたら、戻り値として返し、配列(self.iamges, self.labels)を空にする
                    #print(len(self.images))
                #inputs = np.asarray(self.images,dtype=np.float32) 
                targets = np.asarray(self.labelsx,dtype=np.float32)
                    #print(len(self.images))
                count+=1
                    #if count%1000==0:
                        #print("n*1000!")
            return targets
                    

In [31]:
test_path = "/mnt/Doc/2048pix_tissue_all/data_Split_08_01_01/test_400.csv"
test_datagen = ImageDataGenerator1()
x = test_datagen.flow_from_directory(test_dir,classes)

In [33]:
len(x)

400

<tf.Tensor 'Cast_1:0' shape=(400,) dtype=float32>

In [ ]:
validation_data=test_datagen.flow_from_directory(test_dir, classes),
    validation_steps=120/batchsize,
    callbacks = [modelCheckpoint]